In [ ]:
NAME = "Alexander Lein-McDonough"
CONTRIBUTORS = ""

# **Final Project: Movie Review Sentiment Prediction**

**Overview:**

This project is designed to predict whether the sentiment of a film review is positive or negative based upon the words used in the review.  This project uses a dataset of film reviews from IMDB with two features, the text of the review and the sentiment of the review (0 is negative, 1 is positive).  The model checks which words show up in each review with a Count Vectorizer and uses a Multinomial Naive Bayes model to check if a given review's opinion of the film is positive or not.

**Dataset, Code, and Analysis:**

The follwoing imports the packages and datasets needed to perfrom the algorithm.  The dataset contains only two features, the text of a film review and the sentiment of the review, 0 means the review is negative, 1 means it's positive.  The positive and negative review sentiments are evenly distrobuted within the dataset.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
esw = ENGLISH_STOP_WORDS

reviewData = pd.read_csv(r'clean_data.csv').set_index('Unnamed: 0')
print(reviewData.head(5))

The following defines functions needed to feature engineer our data and to plot our confusion matrix

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
def Series2VectDF(series_data):
    vec = CountVectorizer(ngram_range=(1,1))
    X = vec.fit_transform(series_data)
    return X, vec

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
#     classes = classes[unique_labels((y_true)), (y_pred)))]
    #ME: disable above functionality for the PA
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

Here, we define how many features from our main dataset we use in our calculations with "sentiment" as the text for each review and "y" being our vector for whether the review was positive (1) or negative (0).  We're not able use the entire dataset as it gives a "Memory Error" after attempting to use the Count Vectorizer; so we are forced to use the first 6,000 entries so we do not encounter this type of error.

In [ ]:
n = 6000
sentiment = reviewData['SentimentText'][0:n]
y = reviewData['Sentiment'][0:n]
print(sentiment.head(5))
print(y.head(5))

We convert our data for each comment into an array which shows which words are in each comment (1 if it's in the comment, 0 if it's not) via the columns with a Count Vectorizer, and then removes the words that appear on a list of common english words

In [ ]:
Xd, vecd = Series2VectDF(sentiment)
Xrd = pd.DataFrame(Xd.toarray(), columns=vecd.get_feature_names())
Xrd.drop(Xrd.columns[Xrd.columns.isin(esw)], axis = 1, inplace = True)
print(Xrd.head(5))

We split our count vectorized matrix and our sentement vector and our into training and test data, define an Multinomial Naive Bayes model, and fit the model to our training data.  We used a Multinomial Naive Bayes model as it was able to return give the greatest accuracy of all of the models used on this dataset.  We first considering using Decision Trees, and while it's average accuracy was 81%, we felt that there could be other models.  Next, we tries using Support Vector Machines and only models with the linear kerner were able to sucessfully finish; all other kernels either failed to complete after 12 hours, returned errors, or caused the browser and/or operating system to crash.  We got an accuracy of 83% for SVM with the linear kernel.  We then tries using a few variants of Naive Bayes including Gaussian, Multinomial, and Bernoulli; only Gaussian and Multinomial completed without errors.  We settled on using Multinomial Naive Bayes as it outperformed all of the other algorithms in terms of average accuracy.  We could not try more models due to time constraints.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Xrd, y, random_state=0, test_size=0.2)
model = MultinomialNB()#SVC(kernel = 'linear')
#param_grid = {'kernel': ('linear','poly', 'rbf', 'sigmoid'), 'degree': [1, 3, 5], 'shrinking': [True, False], 'C':[1, 7]}
grid = model#GridSearchCV(model, param_grid, cv=5)
grid.fit(X_train, y_train)
#print(grid.best_params_)
#y_model_test = grid.predict(X_test)
#accuracy_score(y_test, y_model_test)

We run our model on our training and test datasets to see how well the model performs (with accuracy) with "Train Accuracy" being the accuracy of our model on our training data and "Test Accuracy" is the accurracy of our model on our test data.  We also polit confusion matricies, one normalized and one not, to see how our model precdicted each item in the datasets.

In [ ]:
print("Train accuracy is %.2f %%" % (grid.score(X_train, y_train)*100))
print("Test accuracy is %.2f %%" % (grid.score(X_test, y_test)*100))
plot_confusion_matrix(y_train, grid.predict(X_train), y, title = "Training data confusion matrix")
plot_confusion_matrix(y_train, grid.predict(X_train), y, normalize = True, title = "Training data normalized confusion matrix")
plot_confusion_matrix(y_test, grid.predict(X_test), y, title = "Test data confusion matrix")
plot_confusion_matrix(y_test, grid.predict(X_test), y, normalize = True, title = "Test data normalized confusion matrix")

We perform a crossvalidation by chopping the dataset into fifths where each fifth is set to be our training data while the other four fifths become our training data.  We then calculate the accuracies for each run, and get the average and ~98.6% (2 standard deviations) confidence interval for the actuall mean accuracy of our model.

In [ ]:
scores = cross_val_score(model, Xd, y, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

**Summary:**

Using our Multinomial Naive Bayes model, we were able to get 84% Accuracy (With a ~98.6% confidence interval between 81% and 87% Accuracy) for determining whether our review's sentiment was poisitve or negative.  While this model returning 84% accuracy is good, I think there are many ways we could improve this model.  First, we could have the Count Vectorizer include not just individual words, but combinations of multiple words as features; Sometimes the context of what is meant is lost when you just take certain groups of words on their own rather than together.  However, we may have to significantly reduce the size of the data we use for our training and test datasets as not to encounter a memory error. Also, we could use a different method for our classification model that might yield a greater accuracy (I.E. a neural network based model), but it may requre more time and memory resources as not to run into any errors or crashes.

**Sources:**

    Hourrane, Oumaima. “IMDB Movie Reviews for Sentiment Analysis.” Kaggle, Kaggle, 31 July 2018, www.kaggle.com/oumaimahourrane/imdb-reviews. 

    “stop_words.Py.” GitHub, GitHub, 3 Mar. 2012, github.com/scikit-learn/scikit-learn/blob/master/sklearn/feature_extraction/stop_words.py.

    Brownlee, Jason. “Naive Bayes for Machine Learning.” Machine Learning Mastery, 22 Sept. 2016, machinelearningmastery.com/naive-bayes-for-machine-learning/.